In [12]:
import sqlite3 as sql
import pandas as pd 

Apuntador = sql.connect(r'..\DataBase\DataUsers.db')

In [13]:
hd = pd.read_sql_query("SELECT * FROM Aula_fmn279", Apuntador)

In [14]:
hd

,Id_E,Nombre_Estudiante,Correo


In [42]:
Apuntador.execute('''
                    CREATE TABLE IF NOT EXISTS 'Roles' (
                        'Id_Rol' INT PRIMARY KEY,
                        'Rol' TEXT NOT NULL UNIQUE,
                        CHECK(Rol IN("Estudiante", "Profesor")),
                        FOREIGN KEY(Id_Rol) REFERENCES Usuarios_Registrados(Rol)
                        )
                ''')

In [ ]:
Apuntador.execute("INSERT INTO Roles (Id_Rol, Rol) VALUES ('2', 'Profesor')")

In [7]:
a = Apuntador.execute("SELECT MAX(Id) FROM Usuarios_Registrados")
a = a.fetchone()[0]
num = int(a)+1 if (a is not None) else 1
Apuntador.execute(f"INSERT INTO Usuarios_Registrados (Id, Nombre_Usuario, Contraseña, Rol, Email) VALUES ({num},'JoseJose', '12345678', 2, 'a')")

In [2]:
df = pd.read_sql_query("PRAGMA table_info(Usuarios_Registrados)", Apuntador)

In [53]:
Apuntador.execute("DELETE FROM Usuarios_Registrados")

In [7]:
Apuntador.execute("DROP TABLE Usuarios_Registrados")

In [4]:
b = Apuntador.execute("SELECT * FROM 'Aula_qjl-030'")
b = b.fetchall()
print(b)

[(1, 'Jose', 'werttj@gmail.com', 0, 0, 0), (2, 'Felipe', 'wettyj@gmail.com', 0, 0, 0)]


In [11]:
df = pd.read_sql_query("SELECT Usuarios_Registrados.Nombre_Usuario, Roles.Rol FROM Usuarios_Registrados JOIN Roles ON Usuarios_Registrados.Rol = Roles.Id_Rol", Apuntador)

In [5]:
b = Apuntador.execute("PRAGMA table_info('Usuarios_Registrados')")
b = bool(b.fetchall())
print(b)

True


In [3]:
df = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", Apuntador)

In [37]:
b = Apuntador.execute("SELECT name FROM sqlite_master WHERE type='table'")
b = b.fetchall()
for i in range(len(b)):
    Apuntador.execute(f"DROP TABLE '{b[i][0]}'")

#CUIDADO Elimina todas las tablas

OperationalError: database is locked

In [6]:
df = pd.read_sql_query("SELECT * FROM 'Aula_qjl-030';", Apuntador)

In [41]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect(r'..\DataBase\DataUsers.db')

# Create a cursor object to execute SQLite commands
cur = conn.cursor()

# Get a list of all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cur.fetchall()

# Drop each table in the database
for table in tables:
    cur.execute("DROP TABLE IF EXISTS " + table[0])

# Commit the changes
conn.commit()

# Close the cursor and the connection
cur.close()
conn.close()

print("All data in the database has been deleted.")

OperationalError: database is locked

In [13]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect(r'..\DataBase\DataUsers.db')

dc = conn.cursor()

df = dc.execute("SELECT * FROM 'User_Aulas'")

rows = dc.fetchall()

# Display the table
print("Usuarios_Registrados table:")
for row in rows:
    print(row)

conn.commit()
conn.close()

Usuarios_Registrados table:
(1, 3, 1)
(2, 2, 1)


In [8]:
df

In [8]:
Apuntador.commit()
Apuntador.close()